In [49]:
import pickle
import pandas as pd
%run helpers.py
%matplotlib inline

training_file = 'data/train.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

signnames = pd.read_csv('signnames.csv')

n_train = len(X_train)
n_test = len(X_test)
image_shape = X_train.shape[1]
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = 32
Number of classes = 43


In [2]:
print("Training data shape = ", X_train.shape)
print("Number of channels in each image = ", X_train.shape[3])

Training data shape =  (39209, 32, 32, 3)
Number of channels in each image =  3


In [3]:
# Training data
X_train_gray = np.empty( [n_train, image_shape, image_shape], dtype = np.int32)
for i, img in enumerate(X_train):
    X_train_gray[i,:,:] = cv2.equalizeHist(grayscale(img), (0, 254) )
# Test data
X_test_gray = np.empty( [n_test, image_shape, image_shape], dtype = np.int32)
for i, img in enumerate(X_test):
    X_test_gray[i,:,:] = cv2.equalizeHist(grayscale(img), (0, 254) )

In [4]:
from sklearn.model_selection import train_test_split

X_train_gray, X_valid_gray, y_train, y_valid = train_test_split(
    X_train_gray,
    y_train,
    test_size=0.10,
    random_state=1973)

In [5]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
encoder.fit(y_train)
train_labels = encoder.transform(y_train)
valid_labels = encoder.transform(y_valid)
test_labels = encoder.transform(y_test)

# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
train_labels = train_labels.astype(np.float32)
valid_labels = valid_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [6]:
print(X_train_gray.shape)
print(X_valid_gray.shape)
print(X_test_gray.shape)

(35288, 32, 32)
(3921, 32, 32)
(12630, 32, 32)


In [7]:
X_train_gray_flat = flatten_all_gray(X_train_gray)
X_valid_gray_flat = flatten_all_gray(X_valid_gray)
X_test_gray_flat = flatten_all_gray(X_test_gray)

X_train_gray_flat = normalize_grayscale(X_train_gray_flat)
X_valid_gray_flat = normalize_grayscale(X_valid_gray_flat)
X_test_gray_flat = normalize_grayscale(X_test_gray_flat)

In [8]:
import tensorflow as tf
from tensorflow.python.ops.variables import Variable

In [47]:
features_count = 32 * 32
n_hidden_layer = 200
batch_size=100
epochs = 10
learning_rate = 0.01
TRAIN_DIR = 'models/logs/'

# Important!!
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, features_count], name='X')
y = tf.placeholder(tf.float32, [None, n_classes], name='y') 
# Need these when restoring model checkpoint:
tf.add_to_collection("X",X)
tf.add_to_collection("y",y)

train_feed_dict = { X: X_train_gray_flat, y: train_labels}
valid_feed_dict = { X: X_valid_gray_flat, y: valid_labels}
test_feed_dict  = { X: X_test_gray_flat,  y: test_labels}

with tf.name_scope('hidden_layer1'):
    weights_h1 = tf.Variable(tf.random_normal(shape=[features_count, n_hidden_layer], stddev=0.05, dtype=tf.float32), name='weights')
    biases_h1 = tf.Variable(tf.random_normal(shape=[n_hidden_layer], stddev=0.05, dtype=tf.float32), name='biases')
    hidden_layer1 = tf.nn.relu(tf.matmul(X, weights_h1) + biases_h1)
    # For tensorboard
    tf.histogram_summary('hidden_weights', weights_h1)

with tf.name_scope('output'):
    weights_o = tf.Variable(tf.random_normal(shape=[n_hidden_layer, n_classes], stddev=0.05, dtype=tf.float32), name='weights')
    biases_o = tf.Variable(tf.random_normal(shape=[n_classes], stddev=0.05, dtype=tf.float32), name='biases')
    output = tf.matmul(hidden_layer1, weights_o) + biases_o
    tf.add_to_collection('output',output) ## Need this when restoring model checkpoint
    output_probability = tf.nn.softmax(output)
    predicted_class = tf.argmax(output_probability, dimension=1)
    # For tensorboard
    tf.histogram_summary('output', output)

is_correct_prediction = tf.equal(tf.argmax(output_probability, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))
# For tensorboard
tf.scalar_summary("accuracy", accuracy)

cross_entropy = -tf.reduce_sum(y * tf.log(output_probability), reduction_indices=1, name='cross_entropy')
loss = tf.reduce_mean(cross_entropy, name='loss')
# For tensorboard
tf.scalar_summary("loss", loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = optimizer.minimize(loss, global_step=global_step)
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter(TRAIN_DIR)
saver = tf.train.Saver()
init = tf.initialize_all_variables()

In [51]:
#[n.name for n in tf.get_default_graph().as_graph_def().node]

In [48]:
batches = int(math.ceil(len(X_train_gray_flat)/batch_size))

# Clear out tensorboard logs before training
[ os.remove(TRAIN_DIR + '/' + f) for f in os.listdir(TRAIN_DIR) ]

checkpoint_file = os.path.join(TRAIN_DIR, 'checkpoint')

with tf.Session() as sess:
    sess.run(init)
    
    writer.add_graph(sess.graph)
    
    for step in range(epochs):
        
        for batch_i, batch in enumerate(range(batches)):
        
            batch_start = batch_i*batch_size
            batch_features = X_train_gray_flat[batch_start:batch_start + batch_size]
            batch_labels = train_labels[batch_start:batch_start + batch_size]

            _, l = sess.run(
                [train_op, loss],
                feed_dict={X: batch_features, y: batch_labels})

            if not batch_i % 50:
                summary, acc = sess.run([merged, accuracy], feed_dict=valid_feed_dict)    
                writer.add_summary(summary, step)

        # Write a checkpoint
        saver.save(sess, checkpoint_file, global_step = step)